In [1]:
json = {"status":"UP","components":{"diskSpace":{"status":"UP","details":{"total":["java.lang.Long",2046640128],"free":["java.lang.Long",376393728],"threshold":["java.lang.Long",10485760],"exists":true}},"etcd":{"status":"UP"},"failover":{"status":"MASTER"},"ping":{"status":"UP"},"rabbit":{"status":"UP","components":{"ext":{"status":"UP","details":{"version":"3.11.5"}},"int":{"status":"UP","details":{"version":"3.11.5"}}}}}}

NameError: name 'true' is not defined

In [58]:
data = {'status': 'UP', 'components': {'diskSpace': {'status': 'UP', 'details': {'total': ['java.lang.Long', 5150212096], 'free': ['java.lang.Long', 2924032000], 'threshold': ['java.lang.Long', 10485760], 'exists': True}}, 'etcd': {'status': 'UP'}, 'failover': {'status': 'MASTER'}, 'ping': {'status': 'UP'}, 'rabbit': {'status': 'UP', 'components': {'ext': {'status': 'UP', 'details': {'version': '3.11.5'}}, 'int': {'status': 'UP', 'details': {'version': '3.11.5'}}}}}}

In [52]:
import json 
import pandas as pd 
data = pd.json_normalize(data)

data_1 = data.to_json

In [59]:
data

{'status': 'UP',
 'components': {'diskSpace': {'status': 'UP',
   'details': {'total': ['java.lang.Long', 5150212096],
    'free': ['java.lang.Long', 2924032000],
    'threshold': ['java.lang.Long', 10485760],
    'exists': True}},
  'etcd': {'status': 'UP'},
  'failover': {'status': 'MASTER'},
  'ping': {'status': 'UP'},
  'rabbit': {'status': 'UP',
   'components': {'ext': {'status': 'UP', 'details': {'version': '3.11.5'}},
    'int': {'status': 'UP', 'details': {'version': '3.11.5'}}}}}}

In [60]:
from flask import Flask, Response
import json
from prometheus_client import Gauge, generate_latest, REGISTRY
import requests
app = Flask(__name__)


def fetch_and_update_metrics():
    url = "http://127.0.0.1:60378/m7core/health"
    response = requests.get(url)
    data = response.json()
    update_metrics(data)
    return "Metrics updated", 200

# Sample JSON response
json_response = {
    'status': 'UP',
    'components': {
        'diskSpace': {
            'status': 'DOWN',
            'details': {
                'total': ['java.lang.Long', 2042640128],
                'free': ['java.lang.Long', 376393728],
                'threshold': ['java.lang.Long', 10485760],
                'exists': True
            }
        },
        'etcd': {'status': 'DOWN'},
        'failover': {'status': 'MASTER'},
        'ping': {'status': 'DOWN'},
        'rabbit': {
            'status': 'DOWN',
            'components': {
                'ext': {'status': 'UP', 'details': {'version': '3.11.5'}},
                'int': {'status': 'UP', 'details': {'version': '3.11.5'}}
            }
        }
    }
}



try:
    # Prometheus Metrics
    status_gauge = Gauge('system_status', 'Overall system status', ['status'])
    disk_space_total = Gauge('disk_space_total_bytes', 'Total disk space in bytes')
    disk_space_free = Gauge('disk_space_free_bytes', 'Free disk space in bytes')
    disk_space_threshold = Gauge('disk_space_threshold_bytes', 'Disk space threshold in bytes')
    etcd_status = Gauge('etcd_status', 'Status of etcd')
    failover_status = Gauge('failover_status', 'Failover status')
    ping_status = Gauge('ping_status', 'Ping status')
    rabbit_status = Gauge('rabbit_status', 'RabbitMQ status')
    rabbit_ext_version = Gauge('rabbit_ext_version', 'RabbitMQ ext version', ['version'])
    rabbit_int_version = Gauge('rabbit_int_version', 'RabbitMQ int version', ['version'])

except: KeyError

def update_metrics(data):
    status_gauge.labels(status=data['status']).set(1)

    # Update disk space metrics
    disk_space_total.set(data['components']['diskSpace']['details']['total'][1])
    disk_space_free.set(data['components']['diskSpace']['details']['free'][1])
    disk_space_threshold.set(data['components']['diskSpace']['details']['threshold'][1])

    # Update etcd status
    etcd_status.set(1 if data['components']['etcd']['status'] == 'UP' else 0)

    # Update failover status
    failover_status.set(1 if data['components']['failover']['status'] == 'MASTER' else 0)

    # Update ping status
    ping_status.set(1 if data['components']['ping']['status'] == 'UP' else 0)

    # Update rabbit status
    rabbit_status.set(1 if data['components']['rabbit']['status'] == 'UP' else 0)
    rabbit_ext_version.labels(version=data['components']['rabbit']['components']['ext']['details']['version']).set(1)
    rabbit_int_version.labels(version=data['components']['rabbit']['components']['int']['details']['version']).set(1)

# Call update_metrics to initialize the metrics with the sample JSON data
update_metrics(data)

@app.route('/metrics')
def metrics():
    return Response(generate_latest(REGISTRY), mimetype='text/plain')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.30.146.220:8000
Press CTRL+C to quit
127.0.0.1 - - [30/May/2024 15:31:24] "GET /metrics HTTP/1.1" 200 -
